#Library & L2 error

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import math
import itertools

In [ ]:
#L2 error
def L2_error(model,u_exact):
  n = 1000
  x1 = np.random.rand(n,1)
  y1 = np.random.rand(n,1)
  data = np.concatenate((x1,y1),axis = 1)

  u_e = u_exact(data)

  data2 = tf.convert_to_tensor(data)
  u_s = model(data2)
  u_s = u_s.numpy()

  error = math.sqrt(np.sum(np.square(u_e-u_s)))

  L2 = error/n
  return L2

# Create datasets

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
n = 50000
m = 50000
x1 = np.random.rand(n,1)
y1 = np.random.rand(n,1)
data = np.concatenate((x1,y1),axis = 1)

x2_00 = np.zeros((int(m/4),1))
y2_00 = np.random.rand(int(m/4),1)
x2_01 = np.random.rand(int(m/4),1)
y2_01 = np.zeros((int(m/4),1))
x2_10 = np.ones((int(m/4),1))
y2_10 = np.random.rand(int(m/4),1)
x2_11 = np.random.rand(int(m/4),1)
y2_11 = np.ones((int(m/4),1))

data2 = np.concatenate((x2_00,y2_00),axis = 1)
data3 = np.concatenate((x2_01,y2_01),axis = 1)
data2 = np.concatenate((data2,data3),axis = 0)
data3 = np.concatenate((x2_10,y2_10),axis = 1)
data2 = np.concatenate((data2,data3),axis = 0)
data3 = np.concatenate((x2_11,y2_11),axis = 1)
data2 = np.concatenate((data2,data3),axis = 0)

data = np.concatenate((data,data2),axis = 1).astype('float32')
np.random.shuffle(data)

# Model & Train Function

In [ ]:
def build_net(model_layers):
  model = keras.Sequential()
  for i in range(len(model_layers)):
    if i == (len(model_layers)-1):
      model.add(layers.Dense(model_layers[i], activation=None))
    else:
      model.add(layers.Dense(model_layers[i], activation="tanh"))
  return model

In [ ]:
model_layers = [2,50,50,50,50,1]
PINN = build_net(model_layers)

In [ ]:
def grad(X,model):
  with tf.GradientTape(persistent = True) as tape:
    tape.watch(X)
    u = model(X)[:, 0]
  
  u_x = tape.gradient(u,X)[:,0]
  u_y = tape.gradient(u,X)[:,1]

  return u,u_x,u_y

In [ ]:
def exact_u(x):
  PI = math.pi
  u = np.sin(PI*x[0][0])*np.sin(PI*x[0][1])
  return u

In [ ]:
def f(x,y):
  PI = math.pi
  x = tf.convert_to_tensor(x, np.float32)
  y = tf.convert_to_tensor(y, np.float32)
  u = 2*(PI**2)*tf.math.sin(PI*x)*tf.math.sin(PI*y)
  return u

In [ ]:
def train(data,batch_size,model,optimizer,epochs,f):
  if (optimizer == 'adam'):
    optimizer = tf.keras.optimizers.Adam(beta_1 = 0.9,beta_2 = 0.9)
    P_in = data[:,0:2]
    P_on = data[:,2:4]
    for epoch in range(epochs):
      k = int(n/batch_size)
      print("Epoch: "+ str(epoch+1))
      #Load numpy array
      #shuffle & batch

      loss = 0

      for i in range(k):
        P_i = tf.convert_to_tensor(P_in[batch_size*i:batch_size*(i+1),:])
        P_o = tf.convert_to_tensor(P_on[batch_size*i:batch_size*(i+1),:])
        with tf.GradientTape() as tape:
          with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(P_i)
            u,u_x,u_y = grad(P_i,model)
          
          #second-order gradient
          u_xx = tape2.gradient(u_x,P_i)[:,0]
          u_yy = tape2.gradient(u_y,P_i)[:,1]

          #loss in domain
          loss_in = tf.reduce_mean((tf.math.square(u_xx+u_yy+f(P_i[:,0],P_i[:,1]))))

          #loss_on_boundary
          u_on = model(P_o)
          loss_on = tf.reduce_mean(tf.math.square(u_on))

          #total_loss
          total_loss = (loss_in + loss_on)
        
        #parameters update
        grads = tape.gradient(total_loss,model.weights)
        optimizer.apply_gradients(zip(grads,model.weights))

        if (i%10)==0:
          print(total_loss)

    print("Complete!!!")
#  if (optimizer == 'l-bfgs'):


# Training

In [ ]:
batch_size = 64
optimizer = 'adam'
epochs = 50

In [ ]:
train(data,batch_size,PINN,optimizer,epochs,f)

Epoch: 1
tf.Tensor(104.536545, shape=(), dtype=float32)
tf.Tensor(68.539, shape=(), dtype=float32)
tf.Tensor(32.68406, shape=(), dtype=float32)
tf.Tensor(32.44157, shape=(), dtype=float32)
tf.Tensor(31.882404, shape=(), dtype=float32)
tf.Tensor(31.47146, shape=(), dtype=float32)
tf.Tensor(27.677208, shape=(), dtype=float32)
tf.Tensor(31.985888, shape=(), dtype=float32)
tf.Tensor(28.883688, shape=(), dtype=float32)
tf.Tensor(25.802164, shape=(), dtype=float32)
tf.Tensor(27.547516, shape=(), dtype=float32)
tf.Tensor(30.118261, shape=(), dtype=float32)
tf.Tensor(26.599287, shape=(), dtype=float32)
tf.Tensor(26.996048, shape=(), dtype=float32)
tf.Tensor(23.03641, shape=(), dtype=float32)
tf.Tensor(24.247051, shape=(), dtype=float32)
tf.Tensor(23.917503, shape=(), dtype=float32)
tf.Tensor(19.39552, shape=(), dtype=float32)
tf.Tensor(27.614635, shape=(), dtype=float32)
tf.Tensor(22.353746, shape=(), dtype=float32)
tf.Tensor(23.548935, shape=(), dtype=float32)
tf.Tensor(18.084526, shape=(), d

#Prediction

In [ ]:
a = tf.constant([[0.5,0.5]])
print(PINN(a))

tf.Tensor([[1.0125964]], shape=(1, 1), dtype=float32)


In [ ]:
b = tf.constant([[0.,0.]])
print(PINN(b))

tf.Tensor([[-0.07170483]], shape=(1, 1), dtype=float32)


In [ ]:
c = tf.constant([[1.,1.]])
print(PINN(c))

tf.Tensor([[-0.07273927]], shape=(1, 1), dtype=float32)


In [ ]:
print(L2_error(PINN,exact_u))

0.010651841669731637


In [ ]:
b = np.array([[1.,1.]])
print(exact_u(b))

1.4997597826618576e-32
